Create Player Ratings

In [1]:
import pandas as pd
import re
import numpy as np

import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [14]:
player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/data/scored_player_stats.csv")
player_stats['Season'] = player_stats['Match_ID'].apply(lambda x: int(x[:4]))
player_stats['Player_Season'] = player_stats['Player'] + "_" + player_stats['Season'].astype(str)

player_stats['Score'] = player_stats['Behinds'] + player_stats['Goals']*6
player_stats['xScore_Diff'] = player_stats['Score'] - player_stats['xScore']

player_stats['Home_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Away_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[-1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Opponent'] = np.where(player_stats['Team'] == player_stats['Home_Team'], player_stats['Away_Team'], player_stats['Home_Team'])

finals_round_map = {'F1':24,
                    'F2':25,
                    'F3':26,
                    'F4':27}
player_stats['Round_ID'] = player_stats['Round_ID'].astype(str)
player_stats['Round'] = (player_stats['Round_ID'].str[4:].replace(finals_round_map)).astype(int)
player_stats['Round_str'] = player_stats['Round_ID'].str[4:].replace(finals_round_map)
player_stats['Round_ID_num'] = (player_stats['Season'].astype(str) + player_stats['Round'].astype(str)).astype(int)

player_stats.tail()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,Round,Round_str,Round_ID_num
27846,202323_WesternBulldogs_WestCoast,Western Bulldogs,Oskar Baker,202323,Oskar_Baker,MIDFIELDER,CD_I1008855,25,184,87,13,RIGHT,QLD,2017.0,2019.0,Wilston Grange (Qld)/Aspley (NEAFL)/Melbourne,48.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1998-05-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Half Forward Flank Right,FINAL_TEAM,Half-Forward,Forward,NaN,NaN,2023,7.959769,-0.322645,0.016709,0.399113,0.700571,-0.301458,2.852330,3.819445,-0.967115,-0.326746,0.071312,1.048314,3.462521,Oskar Baker_2023,NaN,NaN,Western Bulldogs,West Coast,West Coast,23,23,202323
27847,202323_WesternBulldogs_WestCoast,Western Bulldogs,Rhylee West,202323,Rhylee_West,MEDIUM_FORWARD,CD_I1006127,23,183,82,14,RIGHT,VIC,2018.0,2019.0,Strathmore (Vic)/St Kevin&apos;s College (Vic)...,26.0,fatherSon,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,2000-07-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Forward Pocket Right,FINAL_TEAM,Forward-Pocket,Forward,NaN,NaN,2023,9.770643,0.138877,0.000000,0.569370,0.645190,-0.075820,7.074921,7.476112,-0.401191,0.486820,0.004338,0.926924,6.220417,Rhylee West_2023,NaN,NaN,Western Bulldogs,West Coast,West Coast,23,23,202323
27848,202323_WesternBulldogs_WestCoast,Western Bulldogs,Rory Lobb,202323,Rory_Lobb,KEY_FORWARD,CD_I990740,30,207,106,7,RIGHT,WA,2013.0,2014.0,Bassendean (WA)/Swan Districts (WAFL)/GWS/Frem...,29.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1993-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Forward Pocket Left,FINAL_TEAM,Forward-Pocket,Forward,NaN,NaN,2023,17.116590,-0.143941,-0.002520,1.729477,1.851836,-0.122359,11.169317,12.752323,-1.583006,0.487798,-0.000221,1.150055,10.551092,Rory Lobb_2023,NaN,NaN,Western Bulldogs,West Coast,West Coast,23,23

In [3]:
player_stats[['Position_Group', 'Position_Sub_Group', 'Position']].value_counts().sort_index()

Position_Group  Position_Sub_Group  Position                
Back            Back-Pocket         Back Pocket Left            1178
                                    Back Pocket Right           1167
                Half-Back           Half Back Flank Left        1165
                                    Half Back Flank Right       1157
                Key-Back            Centre Half Back            1159
                                    Full Back                   1181
Forward         Forward-Pocket      Forward Pocket Left         1160
                                    Forward Pocket Right        1169
                Half-Forward        Half Forward Flank Left     1154
                                    Half Forward Flank Right    1145
                Key-Forward         Centre Half Forward         1152
                                    Full Forward                1191
Interchange     Interchange         Interchange                 4651
                                    Substi

Different Positions are expected to do different things

In [15]:
match = player_stats[player_stats['Match_ID'] == "202323_WesternBulldogs_WestCoast"]
match.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,Round,Round_str,Round_ID_num
27805,202323_WesternBulldogs_WestCoast,West Coast,Alex Witherden,202323,Alex_Witherden,MEDIUM_DEFENDER,CD_I998128,24,188,85,23,RIGHT,VIC,2016.0,2017.0,"St Joseph&apos;s, Geel (Vic)/Geel College (Vic...",23.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1998-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Back Pocket Right,FINAL_TEAM,Back-Pocket,Back,NaN,NaN,2023,0.000000,-0.267964,0.023892,1.114284,0.536018,0.578265,8.365421,5.327914,3.037508,-1.010768,0.023430,1.118252,5.976717,Alex Witherden_2023,NaN,NaN,Western Bulldogs,West Coast,Western Bulldogs,23,23,202323
27806,202323_WesternBulldogs_WestCoast,West Coast,Andrew Gaff,202323,Andrew_Gaff,MIDFIELDER,CD_I290801,31,184,83,3,LEFT,VIC,2010.0,2011.0,Kew (Vic)/Carey Grammar (Vic)/Oakleigh U18,4.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1992-06-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wing Left,FINAL_TEAM,Wing,Midfield,NaN,NaN,2023,2.148563,0.191014,0.052414,0.631304,0.340834,0.290469,4.194502,3.959685,0.234816,-0.230022,0.138102,0.953913,7.855344,Andrew Gaff_2023,NaN,NaN,Western Bulldogs,West Coast,Western Bulldogs,23,23,202323
27807,202323_WesternBulldogs_WestCoast,West Coast,Bailey J. Williams,202323,Bailey_J._Williams,RUCK,CD_I1006114,23,201,101,32,RIGHT,VIC,2018.0,2020.0,Pines (Vic)/Western Port SC (Vic)/Dandenong U18,35.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,2000-04-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ruck,FINAL_TEAM,Ruck,Ruck,NaN,NaN,2023,0.000000,0.251066,0.067486,0.449135,0.451447,-0.002312,4.723950,4.275080,0.448870,-0.014674,0.022624,-0.076412,3.294606,Bailey J. Williams_2023,NaN,NaN,Western Bulldogs,West Coast,Western Bulldogs,23,23,202323
27808,202323_WesternBulldogs_We

Team Ratings - Combination of VAEP and Expected VAEP Values should be a correct order (Season by Season)

In [5]:
player_stats.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,Round,Round_str,Round_ID_num
0,202101_BrisbaneLions_Sydney,Brisbane Lions,Brandon Starcevich,202101,Brandon_Starcevich,MEDIUM_DEFENDER,CD_I1002401,22,187,92,37,RIGHT,WA,2017.0,2018.0,Mt Lawley-Inglewood (WA)/Trinity Coll (WA)/Eas...,18.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1999-07-24,84.0,0.0,1.0,0.0,0.0,1.0,1.0,100.0,1.0,1.0,55.6,5.0,0.0,0.0,0.0,4.0,88.9,9.0,31.0,8.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,83.3,0.0,0.0,6.0,2.0,2.0,0.0,1.0,172.0,1.0,5.0,6.1,2.0,0.0,4.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,2.0,4.0,NaN,NaN,Half Back Flank Left,FINAL_TEAM,Half-Back,Back,2021.0,0.0,2021,0.000000,-0.235667,0.000000,0.711460,0.026457,0.685004,3.362637,0.585132,2.777505,-0.461340,0.190552,0.329877,2.328707,Brandon Starcevich_2021,0.0,0.000000,Brisbane Lions,Sydney,Sydney,1,01,20211
1,202101_BrisbaneLions_Sydney,Brisbane Lions,Callum Ah Chee,202101,Callum_Ah_Chee,MEDIUM_DEFENDER,CD_I298437,24,183,79,4,RIGHT,WA,2015.0,2016.0,Kelmscott (WA)/South Fremantle (WAFL)/Gold Coast,8.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1997-10-09,87.0,0.0,0.0,0.0,0.0,4.0,1.0,100.0,1.0,0.0,66.7,8.0,0.0,0.0,0.0,5.0,83.3,12.0,36.0,10.0,5.0,0.0,2.0,0.0,0.0,1.0,0.0,8.0,6.0,0.0,0.0,0.0,0.0,2.0,0.0,7.0,83.3,1.0,1.0,6.0,1.0,0.0,0.0,0.0,258.0,2.0,7.0,8.0,1.0,0.0,2.0,1.0,0.0,2.0,0.0,3.0,1.0,0.0,12.0,6.0,4.0,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,2021.0,0.0,2021,0.000000,-0.075963,0.010892,0.291451,0.479986,-0.188535,1.318957,1.031427,0.287530,-0.487945,-0.039690,0.608221,3.269039,Callum Ah Chee_2021,0.0,0.000000,Brisbane Lions,Sydney,Sydney,1,01,20211
2,202101_BrisbaneLions_Sydney,Brisbane Lions,Charlie Cameron,202101,Charlie_Cameron,MEDIUM_FORWARD,CD_I990609,27,180,75,23,RIGHT,WA,2014.0,2014.0,Newman Saints (WA)/Swan Districts (WAFL)/Adelaide,NaN,rookieElevation,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1994-07-05,77.0,0.0,0.0,9.0,3.0,0.0,0.0,0.0,0.0,0.0,50.0,5.0,0.0,0.0,0.0,3.0,70.0,10.0,39.0,7.0,4.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,57.1,0.0,0.0,7.0,2.3,2.0,1.0,2.0,207.0,1.0,9.0,10.1,0.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,3.0,10.0,0.0,5.0,NaN,NaN,Forward Pocket Right,FINAL_TEAM,Forward-Pocket,Forward,2021.0,0.0,2021,0.000000,0.460322,-0.166355,0.696752,0.681301,0.015451,6.034727,5.882581,0.152147,0.197894,-0.035491,0.859897,8.186274,Charlie Cameron_2021,0.0,0.000000,Brisbane L

In [6]:
sorted(list(set(player_stats['Round_ID'])))[-10:]

['202314',
 '202315',
 '202316',
 '202317',
 '202318',
 '202319',
 '202320',
 '202321',
 '202322',
 '202323']

In [7]:
def create_team_summary(player_stats, rounds = None):
    
    player_stats_copy = player_stats.copy()
    
    if rounds is not None:
        all_rounds = sorted(list(set(player_stats_copy['Round_ID'])))
        rating_rounds = all_rounds[-rounds:]
        player_stats_copy = player_stats_copy[player_stats_copy['Round_ID'].isin(rating_rounds)]
    
    team_for = player_stats_copy.groupby('Team').sum()[['Score', 'xScore','vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
    team_for.columns = [x+"_F" for x in list(team_for)]
    team_against = player_stats_copy.groupby(['Opponent']).sum()[['Score', 'xScore', 'vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
    team_against.columns = [x+"_A" for x in list(team_against)]
    team_games = player_stats_copy.groupby(['Match_ID', 'Team']).size().reset_index().groupby('Team').count()[['Match_ID']]
    team_games.columns = ['Games']
    team_summary = pd.concat([team_games, team_for, team_against], axis=1)
    
    for col in [x.replace("_F", "") for x in list(team_for)]:
        team_summary[col+"_pergame_F"] = team_summary[col+"_F"] / team_summary["Games"]
        team_summary[col+"_pergame_A"] = team_summary[col+"_A"] / team_summary["Games"]

        team_summary[col+"_diff"] = team_summary[col+"_F"] - team_summary[col+"_A"]
        team_summary[col+"_pergame_diff"] = team_summary[col+"_pergame_F"] - team_summary[col+"_pergame_A"]

    return team_summary


In [8]:
team_summary = create_team_summary(player_stats, rounds = None)
team_summary

,Games,Score_F,xScore_F,vaep_value_F,vaep_value_received_F,offensive_value_F,defensive_value_F,exp_vaep_value_F,exp_vaep_value_received_F,exp_offensive_value_F,exp_defensive_value_F,Score_A,xScore_A,vaep_value_A,vaep_value_received_A,offensive_value_A,defensive_value_A,exp_vaep_value_A,exp_vaep_value_received_A,exp_offensive_value_A,exp_defensive_value_A,Score_pergame_F,Score_pergame_A,Score_diff,Score_pergame_diff,xScore_pergame_F,xScore_pergame_A,xScore_diff,xScore_pergame_diff,vaep_value_pergame_F,vaep_value_pergame_A,vaep_value_diff,vaep_value_pergame_diff,vaep_value_received_pergame_F,vaep_value_received_pergame_A,vaep_value_received_diff,vaep_value_received_pergame_diff,offensive_value_pergame_F,offensive_value_pergame_A,offensive_value_diff,offensive_value_pergame_diff,defensive_value_pergame_F,defensive_value_pergame_A,defensive_value_diff,defensive_value_pergame_diff,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_diff,exp_vaep_value_pergame_diff,exp_vaep_value_received_pergame_F,exp_vaep_value_received_pergame_A,exp_vaep_value_received_diff,exp_vaep_value_received_pergame_diff,exp_offensive_value_pergame_F,exp_offensive_value_pergame_A,exp_offensive_value_diff,exp_offensive_value_pergame_diff,exp_defensive_value_pergame_F,exp_defensive_value_pergame_A,exp_defensive_value_diff,exp_defensive_value_pergame_diff
Adelaide,65,5177.0,5327.015157,1163.117796,1056.569166,974.600750,188.517046,7512.177911,6829.311390,6665.435910,846.742001,5506.0,5534.991733,1209.648075,1090.384883,1023.140814,186.507262,7680.391715,7285.439483,6922.945465,757.446250,79.646154,84.707692,-329.0,-5.061538,81.954079,85.153719,-207.976576,-3.199640,17.894120,18.609970,-46.530279,-0.715850,16.254910,16.775152,-33.815717,-0.520242,14.993858,15.740628,-48.540064,-0.746770,2.900262,2.869342,2.009785,0.030920,115.571968,118.159873,-168.213804,-2.587905,105.066329,112.083684,-456.128093,-7.017355,102.545168,106.506853,-257.509556,-3.961685,13.026800,11.653019,89.295751,1.373781
Brisbane Lions,70,6401.0,6366.223908,1355.279634,1151.530897,1163.587168,191.692466,8532.567993,7576.385927,7686.633280,845.934705,5314.0,5351.381915,1203.170038,1120.572976,1020.956274,182.213764,7583.167438,7396.065222,6756.435182,826.732258,91.442857,75.914286,1087.0,15.528571,90.946056,76.448313,1014.841994,14.497743,19.361138,17.188143,152.109596,2.172994,16.450441,16.008185,30.957921,0.442256,16.622674,14.585090,142.630893,2.037584,2.738464,2.603054,9.478703,0.135410,121.893828,108.330963,949.400555,13.562865,108.234085,105.658075,180.320705,2.576010,109.809047,96.520503,930.198098,13.288544,12.084782,11.810461,19.202448,0.274321
Carlton,66,5293.0,5586.511328,1161.951007,1075.374604,980.106645,181.844362,7814.188345,7340.498503,6934.220796,879.967551,5106.0,5215.219343,1141.034959,1066.737533,960.252677,180.782282,7465.125428,6966.109653,6568.582361,896.543062,80.196970,77.363636,187.0,2.833333,84.644111,79.018475,371.291985,5.625636,17.605318,17.288408,20.916048,0.316910,16.293555,16.162690,8.637071,0.130865,14.850101,14.549283,19.853969,0.300818,2.755218,2.739125,1.062079,0.016092,118.396793,113.107961,349.062916,5.288832,111.219674,105.547116,374.388850,5.672558,105.063951,99.523975,365.638435,5.539976,13.332842,13.583986,-16.575512,-0.251144
Collingwood,68,5429.0,5499.727341,1174.501142,1104.395267,993.832755,180.668387,7618.423699,7300.307381,6760.238655,858.185038,5240.0,5310.817666,1158.670549,1046.767498,990.687641,167.982908,7439.361523,7058.687824,6621.100577,818.260941,79.838235,77.058824,189.0,2.779412,80.878343,78.100260,188.909675,2.778083,17.272076,17.039273,15.830594,0.232803,16.241107,15.393640,57.627768,0.847467,14.615188,14.568936,3.145115,0.046252,2.656888,2.470337,12.685479,0.186551,112.035643,109.402375,179.062175,2.633267,107.357461,103.804233,241.619557,3.553229,99.415274,97.369126,139.138079,2.046148,12.620368,12.033249,39.924097,0.587119
Essendon,67,5384.0,5459.636722,1169.427925,1081.394730,1001.503521,167.924404,7568.171912,7116.

In [9]:
def zscale_stats(summary_data, stats):
    
    for col in stats:
        summary_data[col+'_z'] = (summary_data[col] - summary_data[col].mean()) / np.std(summary_data[col])
    summary_data = summary_data.fillna(0)
    
    return summary_data

In [10]:
# rating_factors = ['Score_pergame_diff', 'xScore_pergame_diff', 'vaep_value_pergame_diff', 'exp_vaep_value_pergame_diff']
rating_factors = ['xScore_pergame_diff', 'exp_vaep_value_pergame_diff']
team_ratings = team_summary[rating_factors]
team_ratings = zscale_stats(team_ratings, rating_factors)
team_ratings['team_rating'] = team_ratings[[x+"_z" for x in rating_factors]].mean(axis=1)
team_ratings.sort_values(by = 'team_rating', ascending = False)

,xScore_pergame_diff,exp_vaep_value_pergame_diff,xScore_pergame_diff_z,exp_vaep_value_pergame_diff_z,team_rating
Melbourne,19.648842,18.662259,1.386038,1.341748,1.363893
Geelong,15.661688,15.382236,1.109183,1.109725,1.109454
Brisbane Lions,14.497743,13.562865,1.028363,0.981026,1.004694
Western Bulldogs,12.324383,12.998722,0.877451,0.941119,0.909285
Port Adelaide,10.229226,10.854797,0.731970,0.789462,0.760716
Sydney,8.232568,7.964706,0.593329,0.585022,0.589175
Carlton,5.625636,5.288832,0.412312,0.395735,0.404024
Richmond,3.930069,3.468487,0.294577,0.266967,0.280772
Collingwood,2.778083,2.633267,0.214587,0.207885,0.211236
St Kilda,0.324976,0.796887,0.044251,0.077983,0.061117


In [11]:
def create_team_ratings(team_summary, rating_factors = []):
    
    team_ratings = team_summary[rating_factors]
    team_ratings = zscale_stats(team_ratings, rating_factors)
    team_ratings['team_rating'] = team_ratings[[x+"_z" for x in rating_factors]].mean(axis=1)
    
    return team_ratings.sort_values(by = 'team_rating', ascending = False)

In [19]:
player_stats_2023 = player_stats[player_stats['Season'] == 2023]
team_summary = create_team_summary(player_stats_2023, rounds = None)
team_ratings = create_team_ratings(team_summary, rating_factors = ['Score_pergame_diff', 'xScore_pergame_diff', 'vaep_value_pergame_diff', 'exp_vaep_value_pergame_diff'])
team_ratings

,Score_pergame_diff,xScore_pergame_diff,vaep_value_pergame_diff,exp_vaep_value_pergame_diff,Score_pergame_diff_z,xScore_pergame_diff_z,vaep_value_pergame_diff_z,exp_vaep_value_pergame_diff_z,team_rating
Melbourne,16.761905,13.174047,2.744222,12.537216,0.939246,0.813727,1.052393,0.791532,0.899225
Brisbane Lions,15.238095,15.840140,1.933281,13.935466,0.853758,0.979036,0.741078,0.880072,0.863486
Collingwood,17.238095,10.855212,2.729729,9.952910,0.965960,0.669950,1.046829,0.627890,0.827657
Port Adelaide,8.809524,15.578198,0.945720,16.150295,0.493110,0.962794,0.361961,1.020318,0.709546
Adelaide,13.272727,9.869886,1.877318,11.106035,0.743499,0.608856,0.719595,0.700908,0.693214
Carlton,11.409091,10.237749,1.707680,10.317848,0.638948,0.631665,0.654472,0.650998,0.644021
Geelong,11.818182,5.167031,1.849428,5.649172,0.661898,0.317260,0.708888,0.355370,0.510854
St Kilda,6.380952,9.217302,1.008107,9.607024,0.356864,0.568393,0.385911,0.605988,0.479289
Sydney,8.047619,6.446398,1.107613,6.016425,0.450366,0.396586,0.424111,0.378626,0.412422
Western Bulldogs,6.181818,6.049869,0.763687,6.333334,0.345693,0.371999,0.292080,0.398693,0.352116


In [30]:
def get_cumulative_round_summary(player_stats, current_round_id, rating_factor, num_rounds = None):
    
    current_season = int(current_round_id[:4])
    if "F" in current_round_id:
        current_round_id_int = int(finals_round_map[current_round_id[4:]])
        current_round_id_str = finals_round_map[current_round_id[4:]]
    else:
        current_round_id_int = int(current_round_id[4:])
        current_round_id_str = current_round_id[4:]
    current_round_id_num = int(str(current_season) + str(current_round_id_str))

    past_seasons_player_stats = player_stats[(player_stats['Season'] < current_season)]
    current_round_player_stats = player_stats[(player_stats['Season'] == current_season) & (player_stats['Round'] <= current_round_id_int)]
    rating_player_stats = pd.concat([past_seasons_player_stats, current_round_player_stats], axis=0)

    round_summary_dict = {}
    for round_id in list(set(rating_player_stats['Round_ID'])):
        season = int(round_id[:4])
        if "F" in round_id:
            round_id_int = int(finals_round_map[round_id[4:]])
            round_id_str = finals_round_map[round_id[4:]]
        else:
            round_id_int = int(round_id[4:])
            round_id_str = round_id[4:]
        seasons_player_stats = rating_player_stats[(rating_player_stats['Season'] < season)]
        round_player_stats = rating_player_stats[(rating_player_stats['Season'] == season) & (rating_player_stats['Round'] <= round_id_int)]
        latest_round_player_stats = pd.concat([seasons_player_stats, round_player_stats], axis=0)
        
        latest_round_team_summary = create_team_summary(latest_round_player_stats, rounds = num_rounds)
        
        round_summary_dict[int(str(season) + str(round_id_str))] = latest_round_team_summary[rating_factor]
        
    round_summary_df = pd.DataFrame.from_dict(round_summary_dict)
    # round_summary_df.T.sort_index()
    round_summary_df = round_summary_df.reindex(sorted(round_summary_df.columns), axis=1).sort_values(by=int(current_round_id_num), ascending=False)
    return round_summary_df

In [31]:
xscore_f_summary = get_cumulative_round_summary(player_stats, current_round_id='202323', rating_factor = 'xScore_pergame_F', num_rounds = 10)
# xscore_f_summary[[x for x in list(xscore_f_summary) if '2023' in str(x)]]
xscore_f_summary

,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112,202113,202114,202115,202116,202117,202118,202119,202120,202121,202122,202123,202124,202125,202126,202127,202201,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212,202213,202214,202215,202216,202217,202218,202219,202220,202221,202222,202223,202224,202225,202226,202227,202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311,202312,202313,202314,202315,202316,202317,202318,202319,202320,202321,202322,202323
Carlton,84.197005,80.916275,94.170831,96.014360,93.691386,92.292777,94.283868,92.814433,89.829921,88.161757,87.109941,87.897614,84.255133,82.211379,83.213148,81.958491,77.824745,79.736297,81.566776,82.265521,80.437207,78.765945,78.648312,78.147995,76.239568,76.558390,77.813672,78.813835,77.337026,75.340948,78.758755,82.438127,79.202410,86.063949,88.956862,90.245759,91.121945,86.708798,88.168966,88.960490,88.780163,87.554606,92.523238,91.133704,85.118728,85.698579,82.582922,84.542328,84.024997,84.761493,85.723001,86.248734,83.870981,78.728247,79.476105,77.220654,80.020944,82.947058,79.640493,79.326902,87.645736,85.101119,82.682512,80.950374,81.640782,76.907157,76.404238,78.988193,81.150256,83.314718,79.006424,84.089353,91.579918,93.635301,93.178419,94.388845,95.620636
Sydney,92.468489,96.490469,102.182044,100.628085,95.712307,86.909251,85.298589,82.317101,83.265167,81.540361,80.717459,80.438926,73.983516,71.540994,70.882156,77.890592,79.667315,81.484403,82.895318,85.080151,82.337013,81.248963,88.371843,88.694392,90.758253,88.856797,88.461322,91.668544,88.478048,84.456936,86.939191,89.173243,90.319563,89.318182,86.615880,88.193400,86.663419,85.603699,85.840392,88.985678,88.226266,86.307723,83.309017,86.548525,87.633269,88.850810,93.799860,97.231192,97.384158,97.617142,98.233830,99.559127,99.106501,91.211632,94.132920,95.032392,91.792972,87.844194,88.204958,81.170510,84.447583,82.925773,80.597568,85.499216,81.659712,76.883842,72.774303,70.361320,83.511043,90.129111,86.172077,87.509136,89.017708,89.453630,89.816930,90.276809,93.270885
Brisbane Lions,69.728083,76.918707,77.877148,71.201923,78.109112,80.580829,83.019726,85.769847,88.494462,89.130849,93.661811,91.932190,93.280875,97.650889,96.650696,96.558666,94.348315,92.964967,95.600101,94.211471,94.321791,99.776944,101.780185,98.812394,97.518297,98.119091,100.663984,100.050570,93.155466,100.662080,94.289952,91.524391,88.661559,92.896551,93.951944,92.690686,92.831567,94.961683,95.449535,91.028838,93.914979,89.513752,90.050373,89.147433,90.052309,90.711345,91.845605,91.029563,91.531172,87.403967,87.993514,90.361435,86.051300,85.992340,80.539173,80.273236,75.574758,76.544266,75.082328,80.178521,81.655258,83.061920,88.403869,90.168016,93.879860,94.847528,96.276908,95.959435,94.860025,97.162897,98.730982,98.810089,94.648051,91.310045,90.350962,90.294019,92.532492
Adelaide,NaN,108.378634,99.162397,98.131887,90.098748,91.609992,88.482220,81.910835,82.124786,82.253137,82.445489,79.407239,78.303426,76.329267,77.366000,74.788185,70.733286,72.140082,72.283369,70.412942,67.509906,65.991860,70.661352,70.661352,70.077779,69.452515,74.969722,79.566462,76.310992,79.279103,85.479533,87.913017,82.335251,79.562066,77.389196,76.337936,77.601022,75.607084,77.661299,76.928692,75.555070,78.161808,74.910962,74.212102,75.682634,76.165626,76.824121,78.368501,79.536171,77.393178,76.782496,73.580899,75.776848,78.967455,82.702821,85.221361,86.509384,88.877042,90.790629,94.926008,92.019632,89.023677,90.826187,87.141269,85.749490,85.438132,89.338281,89.165855,86.188439,91.081282,90.936579,90.286817,88.793229,95.075961,97.640794,99.412688,92.204167
Port Adelaide,115.529080,107.691528,93.419407,93.735986,92.188816,91.187074,86.660801,88.115299,85.973642,83.180476,81.235799,79.167113,80.916673,79.261435,78.712765,79.061282,78.188233,76.289045,79.846014,82.698142,78.773756,84.987123,84.195671,85.293310,85.763443,83.061409,86.988369,85.144635,82.062787,83.305801,81.259383,73.774160,76.9

In [26]:
xscore_a_summary = get_cumulative_round_summary(player_stats, current_round_id='202323', rating_factor = 'xScore_pergame_A', num_rounds = 10).sort_values(by=202323, ascending=True)
xscore_a_summary[[x for x in list(xscore_a_summary) if '2023' in str(x)]]

,202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311,202312,202313,202314,202315,202316,202317,202318,202319,202320,202321,202322,202323
Carlton,79.771067,81.703919,77.671763,75.450737,80.259048,82.948715,77.053212,78.559869,77.278996,78.431750,77.522278,78.726952,79.737046,77.727323,74.373480,71.119878,73.315729,73.278618,72.665467,73.391364,71.101399,68.265446,68.850166
Greater Western Sydney,92.341902,90.304829,85.360394,94.846260,101.179360,99.662573,100.131437,96.028495,97.310171,97.469410,95.045491,96.613381,94.672178,88.004902,88.004902,85.074210,82.207477,81.184509,74.927045,73.038702,72.645079,72.494159,69.909033
St Kilda,81.540533,80.826323,80.909635,76.510405,74.164532,70.235261,72.380789,71.298053,75.541579,74.668005,78.985497,82.362419,80.156755,81.732895,82.214212,79.652598,77.094236,78.562087,73.765786,75.075582,69.924863,70.666635,70.887670
Brisbane Lions,91.608441,89.181519,85.096605,85.832793,87.500701,86.492561,84.144595,81.510593,77.507384,75.939288,70.351674,70.883900,72.702694,70.181538,70.036686,67.135750,63.581061,64.614032,64.448330,68.377362,67.211581,70.512372,71.526782
Melbourne,73.054997,72.275213,76.524568,75.143025,78.522636,81.313043,77.670349,78.256225,74.817222,77.436716,77.809127,73.685939,71.124953,71.246452,68.084884,63.661411,65.976802,66.785496,71.729804,69.938187,71.159706,72.370132,73.582596
Western Bulldogs,78.941552,77.230408,72.952820,73.014992,77.647570,78.560288,76.501663,76.013110,74.604282,72.541554,70.183265,70.563301,74.255206,73.950197,72.925941,74.377558,76.122389,77.254787,75.980414,79.790788,79.437895,79.018421,77.253497
Sydney,73.041837,72.664146,79.609620,77.583914,80.665672,84.254658,85.798049,85.200973,87.181869,84.299341,84.979726,86.992746,83.843353,88.174002,82.445222,78.247574,80.400577,80.018905,76.337439,77.131474,78.882999,78.952395,78.421891
Adelaide,94.329530,90.593858,92.603015,92.092489,87.371520,82.355433,79.209497,80.102288,79.665021,82.195224,79.932683,80.988155,77.915530,78.653281,81.678417,79.488047,84.268438,82.898618,85.548788,81.795707,79.125506,77.999887,81.672035
Fremantle,74.658469,73.948526,73.318907,74.219090,79.283186,84.389571,87.730875,84.259263,82.577724,82.532562,83.951831,86.531893,85.366078,86.801093,84.795079,80.624691,79.921443,84.651256,86.159828,85.313907,84.488645,80.914380,81.819682
Collingwood,81.974677,80.351317,78.347849,79.970826,79.347419,77.721359,76.418318,77.588789,75.311826,74.206194,72.873144,71.697130,75.461692,71.101204,72.213867,70.829923,71.208002,71.954239,73.346031,75.401414,78.112270,81.813738,82.623498


In [ ]:
get_cumulative_round_summary(player_stats, current_round_id='202320', rating_factor = 'exp_vaep_value_pergame_diff', num_rounds=10)

In [ ]:
def get_cumulative_round_team_ratings(player_stats, current_round_id, rating_factors, num_rounds = None):
    
    current_season = int(current_round_id[:4])
    if "F" in current_round_id:
        current_round_id_int = int(finals_round_map[current_round_id[4:]])
        current_round_id_str = finals_round_map[current_round_id[4:]]
    else:
        current_round_id_int = int(current_round_id[4:])
        current_round_id_str = current_round_id[4:]
    current_round_id_num = int(str(current_season) + str(current_round_id_str))

    past_seasons_player_stats = player_stats[(player_stats['Season'] < current_season)]
    current_round_player_stats = player_stats[(player_stats['Season'] == current_season) & (player_stats['Round'] <= current_round_id_int)]
    rating_player_stats = pd.concat([past_seasons_player_stats, current_round_player_stats], axis=0)

    round_summary_dict = {}
    for round_id in list(set(rating_player_stats['Round_ID'])):
        season = int(round_id[:4])
        if "F" in round_id:
            round_id_int = int(finals_round_map[round_id[4:]])
            round_id_str = finals_round_map[round_id[4:]]
        else:
            round_id_int = int(round_id[4:])
            round_id_str = round_id[4:]
        seasons_player_stats = rating_player_stats[(rating_player_stats['Season'] < season)]
        round_player_stats = rating_player_stats[(rating_player_stats['Season'] == season) & (rating_player_stats['Round'] <= round_id_int)]
        latest_round_player_stats = pd.concat([seasons_player_stats, round_player_stats], axis=0)
        
        latest_round_team_summary = create_team_summary(latest_round_player_stats, rounds = num_rounds)
        latest_round_team_ratings = create_team_ratings(latest_round_team_summary, rating_factors = rating_factors)
        round_summary_dict[int(str(season) + str(round_id_str))] = latest_round_team_ratings['team_rating']
        
    round_summary_df = pd.DataFrame.from_dict(round_summary_dict)
    round_summary_df = round_summary_df.reindex(sorted(round_summary_df.columns), axis=1).sort_values(by=int(current_round_id_num), ascending=False)
    return round_summary_df

In [ ]:
get_cumulative_round_team_ratings(player_stats, current_round_id='202323', rating_factors = ['xScore_pergame_diff', 'exp_vaep_value_pergame_diff'], num_rounds=10)